### Residual Neural Network

In [4]:
import tensorflow as tf
import numpy as np

##### Hyper parameters

In [5]:
EPOCHS = 10

##### Resisual Unit

In [20]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        # identity를 정함
        if filter_in == filter_out :
            self.identity = lambda x: x
        else :
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')
    
    def __call__(self, x, training=False, mask=None):
        h = self.bn1(x, training=training) # batch norm은 training 시와 test시 달라져야 한다.
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h

#### Residual Layer 구현

In [21]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = list()
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            # [16] + [32, 32, 32]
            # zip([16, 32, 32, 32], [32, 32, 32])
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x
        

#####  모델 정의

In [22]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        # 처음에 feature를 한번 뽑아줘야 함.
        self.conv1 = tf.keras.layers.Conv2D(8,(3,3), padding='same', activation='relu' ) # 28x28x8
        
        self.res1 = ResnetLayer(8, (16,16), (3,3)) # 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2,2)) # 14x14x16
        
        self.res2 = ResnetLayer(16, (32,32), (3,3)) # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2,2)) # 7x7x32
        
        self.res3 = ResnetLayer(32, (64,64), (3,3)) # 7x7x64
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

### 학습, 테스트 루프 정의

In [23]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

#### 데이터 셋 준비

In [24]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 모델 생성

In [25]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

##### 학습 루프 동작

In [28]:
    for epoch in range(EPOCHS):
        for images, labels in train_ds:
            train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        for test_images, test_labels in test_ds:
            test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result() * 100,
                              test_loss.result(),
                              test_accuracy.result()*100))

Epoch 1, Loss: 0.10551720857620239, Accuracy: 97.0574951171875, Test Loss: 0.04999997466802597, Test Accuracy: 98.62999725341797
Epoch 2, Loss: 0.08757072687149048, Accuracy: 97.55833435058594, Test Loss: 0.043274760246276855, Test Accuracy: 98.73500061035156
Epoch 3, Loss: 0.07618454843759537, Accuracy: 97.8862533569336, Test Loss: 0.04000457748770714, Test Accuracy: 98.80333709716797
Epoch 4, Loss: 0.0677381157875061, Accuracy: 98.11299896240234, Test Loss: 0.045524075627326965, Test Accuracy: 98.7249984741211
Epoch 5, Loss: 0.061346136033535004, Accuracy: 98.28916931152344, Test Loss: 0.047910578548908234, Test Accuracy: 98.65599822998047
Epoch 6, Loss: 0.056704096496105194, Accuracy: 98.4173812866211, Test Loss: 0.04712387174367905, Test Accuracy: 98.6866683959961
Epoch 7, Loss: 0.05257628113031387, Accuracy: 98.5320816040039, Test Loss: 0.04496166482567787, Test Accuracy: 98.76714324951172
Epoch 8, Loss: 0.04929240792989731, Accuracy: 98.6253662109375, Test Loss: 0.042724985629320